In [3]:
import slopestabilitytools
import settings
import os
settings.init()

In [4]:
print(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/')
test_results = slopestabilitytools.datamanagement.import_tests(abs_path=os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/')
#for test_name in test_results.keys():


/home/felikskrno/Documents/Studies/MasterThesis/PythonProjects/SlopeStability/
